<a href="https://colab.research.google.com/github/honganhss/OpenDataFitHou/blob/main/OverpassApi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thử API có hoạt hay không:

In [2]:
import requests
import pandas as pd

# =============================
# Cấu hình
# =============================
lat_min, lat_max = 20.9, 21.2
lon_min, lon_max = 105.7, 106.0

lat_step = (lat_max - lat_min) / 3
lon_step = (lon_max - lon_min) / 3

# API endpoint
base_url = "https://overpass.kumi.systems/api/interpreter"

# =============================
# Test với ô (0,0) - góc dưới bên trái
# =============================
i, j = 0, 0
bbox = (
    lat_min + i*lat_step,
    lon_min + j*lon_step,
    lat_min + (i+1)*lat_step,
    lon_min + (j+1)*lon_step
)
bbox_str = f"({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]})"

print(f"🔍 Đang lấy dữ liệu bus stop cho bbox: {bbox_str}")

# Query Overpass
query = f"""
[out:json][timeout:120];
(
  node["highway"="bus_stop"]{bbox_str};
);
out body;
"""

try:
    response = requests.get(base_url, params={'data': query})
    response.raise_for_status()
    data = response.json()
    print(f"✅ API trả về {len(data.get('elements', []))} đối tượng")
except Exception as e:
    print("❌ Lỗi khi gọi Overpass API:", e)
    data = {"elements": []}

# =============================
# Chuyển dữ liệu sang DataFrame
# =============================
all_data = []
for el in data.get("elements", []):
    tags = el.get("tags", {})
    all_data.append({
        "id": el.get("id"),
        "type": el.get("type"),
        "name": tags.get("name"),
        "lat": el.get("lat"),
        "lon": el.get("lon"),
    })

df = pd.DataFrame(all_data)

# Hiển thị kết quả
if not df.empty:
    print("📋 5 dòng dữ liệu đầu tiên:")
    print(df.head())
else:
    print("⚠️ Không tìm thấy dữ liệu trong bbox này")


🔍 Đang lấy dữ liệu bus stop cho bbox: (20.9,105.7,21.0,105.8)
✅ API trả về 337 đối tượng
📋 5 dòng dữ liệu đầu tiên:
          id  type                                               name  \
0  444394880  node             Đại học Kiến Trúc - Trần Phú (Hà Đông)   
1  444394950  node  Học viện công nghệ bưu chính viễn thông - Trần...   
2  444394956  node                Số 10 Trần Phú - Siêu thị Việt Long   
3  444394979  node  216 Nguyễn Trãi - Nhà Văn hóa tổ dân phố số 8 ...   
4  738814236  node  Đối diện Trung tâm Văn hóa Thông tin Tỉnh Hà T...   

         lat         lon  
0  20.981091  105.789224  
1  20.980523  105.788002  
2  20.983357  105.791540  
3  20.987886  105.797790  
4  20.972302  105.782531  


Cấu Hình Chung


In [8]:
from google.colab import userdata

# Lấy token từ Colab secrets
token = userdata.get('GITHUB_TOKEN')

# Đưa vào biến môi trường (để script khác có thể dùng)
import os
os.environ['GITHUB_TOKEN'] = token

print("✅ Đã load token, độ dài:", len(token) if token else "Không có")

!git ls-remote https://${GITHUB_TOKEN}@github.com/honganhss/OpenDataFitHou.git



✅ Đã load token, độ dài: 40
f1dfbfb7ac0ed7b6f7652e489c7f4c2fd6d717bb	HEAD
f1dfbfb7ac0ed7b6f7652e489c7f4c2fd6d717bb	refs/heads/main
f1dfbfb7ac0ed7b6f7652e489c7f4c2fd6d717bb	refs/tags/v0.1


In [3]:
import requests, json, time, subprocess, os
from pathlib import Path
from datetime import datetime

# Hà Nội bounding box
LAT_MIN, LAT_MAX = 20.9, 21.2
LON_MIN, LON_MAX = 105.7, 106.0

N_LAT, N_LON = 3, 3
lat_step = (LAT_MAX - LAT_MIN) / N_LAT
lon_step = (LON_MAX - LON_MIN) / N_LON

# Tạo danh sách 9 bbox (lat_min, lon_min, lat_max, lon_max)
BBOXES = []
for i in range(N_LAT):
    for j in range(N_LON):
        bbox = (
            LAT_MIN + i*lat_step,
            LON_MIN + j*lon_step,
            LAT_MIN + (i+1)*lat_step,
            LON_MIN + (j+1)*lon_step
        )
        BBOXES.append(bbox)

print("📦 Đã tạo 9 bbox cho Hà Nội:")
for b in BBOXES:
    print(b)

📦 Đã tạo 9 bbox cho Hà Nội:
(20.9, 105.7, 21.0, 105.8)
(20.9, 105.8, 21.0, 105.9)
(20.9, 105.9, 21.0, 106.0)
(21.0, 105.7, 21.099999999999998, 105.8)
(21.0, 105.8, 21.099999999999998, 105.9)
(21.0, 105.9, 21.099999999999998, 106.0)
(21.099999999999998, 105.7, 21.2, 105.8)
(21.099999999999998, 105.8, 21.2, 105.9)
(21.099999999999998, 105.9, 21.2, 106.0)


In [9]:
SERVERS = [
    "https://overpass.kumi.systems/api/interpreter"
]
TIMEOUT = 120
SLEEP = 1.0

In [25]:
GITHUB_REPO = "honganhss/OpenDataFitHou"
GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN")

In [22]:
OUT_DIR = Path("opendata_hanoi")
PATH_IN_REPO = "opendata_hanoi/bus_stop.geojson"
OUT_DIR.mkdir(exist_ok=True)

Hàm Tiện Ích

In [26]:
import requests
import base64
from pathlib import Path
from datetime import datetime, timezone

now = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S UTC")

def query_overpass(q):
    for srv in SERVERS:
        try:
            resp = requests.get(srv, params={"data": q}, timeout=TIMEOUT)
            resp.raise_for_status()
            return resp.json()
        except Exception as e:
            print(f"⚠️ Lỗi server {srv}: {e}")
            time.sleep(2)
    return {"elements":[]}

def element_latlon(el):
    lat, lon = el.get("lat"), el.get("lon")
    if lat is None or lon is None:
        c = el.get("center") or {}
        lat, lon = c.get("lat"), c.get("lon")
    return lat, lon

def push_to_github(local_file, repo=GITHUB_REPO, token=GITHUB_TOKEN, branch="main"):
    if not token:
        print("❌ Chưa có GITHUB_TOKEN, bỏ qua push.")
        return
    local_file = Path(local_file)
    if not local_file.exists():
        print(f"❌ File {local_file} không tồn tại.")
        return

    # Đường dẫn file trong repo
    repo_path = f"data/{local_file.name}"

    # Đọc file và encode base64
    with open(local_file, "rb") as f:
        content_b64 = base64.b64encode(f.read()).decode()

    # GitHub API URL
    url = f"https://api.github.com/repos/{repo}/contents/{repo_path}"

    # Kiểm tra xem file đã tồn tại chưa để lấy sha (cần khi update)
    headers = {"Authorization": f"token {token}"}
    resp = requests.get(url, headers=headers, params={"ref": branch})
    if resp.status_code == 200:
        sha = resp.json()["sha"]
    else:
        sha = None

    # Commit message
    now = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")
    message = f"Update {local_file.name} {now}"

    data = {
        "message": message,
        "content": content_b64,
        "branch": branch,
    }
    if sha:
        data["sha"] = sha  # cần khi update file đã tồn tại

    # Gửi request PUT để tạo hoặc update file
    resp = requests.put(url, headers=headers, json=data)
    if resp.status_code in [200, 201]:
        print(f"✅ Đã push file {local_file.name} lên GitHub repo {repo} (branch {branch})")
    else:
        print("❌ Lỗi khi push lên GitHub:", resp.status_code, resp.text)

In [14]:
!git -C repo_push config user.name "honganhss"
!git -C repo_push config user.email "honganh068204@gmail.com"


In [16]:
features = []
seen = set()

for bbox in BBOXES:
    bbox_str = f"({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]})"
    print(f"🔍 Query bus_stop tại bbox {bbox_str}")
    q = f"""
[out:json][timeout:{TIMEOUT}];
(
  node["highway"="bus_stop"]{bbox_str};
  way["highway"="bus_stop"]{bbox_str};
  relation["highway"="bus_stop"]{bbox_str};
);
out center;
"""
    data = query_overpass(q)
    for el in data.get("elements", []):
        key = (el["type"], el["id"])
        if key in seen:
            continue
        seen.add(key)
        lat, lon = element_latlon(el)
        if lat is None or lon is None:
            continue
        tags = el.get("tags",{})
        feat = {
            "type":"Feature",
            "geometry":{"type":"Point","coordinates":[lon,lat]},
            "properties":{
                "osm_id":el["id"],
                "osm_type":el["type"],
                "name":tags.get("name"),
                "tags":tags
            }
        }
        features.append(feat)
    time.sleep(SLEEP)

geo = {"type":"FeatureCollection","features":features}
out_file = OUT_DIR/"bus_stop.geojson"
with open(out_file,"w",encoding="utf-8") as f:
    json.dump(geo,f,ensure_ascii=False,indent=2)

print(f"💾 Lưu {out_file} với {len(features)} điểm bus_stop")


🔍 Query bus_stop tại bbox (20.9,105.7,21.0,105.8)
🔍 Query bus_stop tại bbox (20.9,105.8,21.0,105.9)
🔍 Query bus_stop tại bbox (20.9,105.9,21.0,106.0)
🔍 Query bus_stop tại bbox (21.0,105.7,21.099999999999998,105.8)
🔍 Query bus_stop tại bbox (21.0,105.8,21.099999999999998,105.9)
🔍 Query bus_stop tại bbox (21.0,105.9,21.099999999999998,106.0)
🔍 Query bus_stop tại bbox (21.099999999999998,105.7,21.2,105.8)
🔍 Query bus_stop tại bbox (21.099999999999998,105.8,21.2,105.9)
🔍 Query bus_stop tại bbox (21.099999999999998,105.9,21.2,106.0)
💾 Lưu opendata_hanoi/bus_stop.geojson với 2749 điểm bus_stop


In [21]:
from IPython.display import FileLink

display(FileLink(out_file))


/content/opendata_hanoi/bus_stop.geojson

In [27]:
# Push lên GitHub
push_to_github(out_file)

/tmp/ipython-input-1839916761.py:54: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


✅ Đã push file bus_stop.geojson lên GitHub repo honganhss/OpenDataFitHou (branch main)


In [29]:
features = []
seen = set()

for bbox in BBOXES:
    bbox_str = f"({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]})"
    print(f"🔍 Query sân chơi tại bbox {bbox_str}")
    q = f"""
[out:json][timeout:{TIMEOUT}];
(
  node["leisure"="playground"]{bbox_str};
  way["leisure"="playground"]{bbox_str};
  relation["leisure"="playground"]{bbox_str};
);
out center;
"""
    data = query_overpass(q)
    for el in data.get("elements", []):
        key = (el["type"], el["id"])
        if key in seen:
            continue
        seen.add(key)
        lat, lon = element_latlon(el)
        if lat is None or lon is None:
            continue
        tags = el.get("tags",{})
        feat = {
            "type":"Feature",
            "geometry":{"type":"Point","coordinates":[lon,lat]},
            "properties":{
                "osm_id":el["id"],
                "osm_type":el["type"],
                "name":tags.get("name"),
                "tags":tags
            }
        }
        features.append(feat)
    time.sleep(SLEEP)

geo = {"type":"FeatureCollection","features":features}
out_file = OUT_DIR/"playground.geojson"
with open(out_file,"w",encoding="utf-8") as f:
    json.dump(geo,f,ensure_ascii=False,indent=2)

print(f"💾 Lưu {out_file} với {len(features)} sân chơi.")


🔍 Query sân chơi tại bbox (20.9,105.7,21.0,105.8)
🔍 Query sân chơi tại bbox (20.9,105.8,21.0,105.9)
⚠️ Lỗi server https://overpass.kumi.systems/api/interpreter: Expecting value: line 1 column 1 (char 0)
🔍 Query sân chơi tại bbox (20.9,105.9,21.0,106.0)
🔍 Query sân chơi tại bbox (21.0,105.7,21.099999999999998,105.8)
🔍 Query sân chơi tại bbox (21.0,105.8,21.099999999999998,105.9)
🔍 Query sân chơi tại bbox (21.0,105.9,21.099999999999998,106.0)
🔍 Query sân chơi tại bbox (21.099999999999998,105.7,21.2,105.8)
⚠️ Lỗi server https://overpass.kumi.systems/api/interpreter: Expecting value: line 1 column 1 (char 0)
🔍 Query sân chơi tại bbox (21.099999999999998,105.8,21.2,105.9)
🔍 Query sân chơi tại bbox (21.099999999999998,105.9,21.2,106.0)
💾 Lưu opendata_hanoi/playground.geojson với 136 sân chơi.


In [30]:
# Push lên GitHub
push_to_github(out_file)

/tmp/ipython-input-1839916761.py:54: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


✅ Đã push file playground.geojson lên GitHub repo honganhss/OpenDataFitHou (branch main)


In [31]:
features = []
seen = set()

for bbox in BBOXES:
    bbox_str = f"({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]})"
    print(f"🔍 Query toilets công cộng tại bbox {bbox_str}")
    q = f"""
[out:json][timeout:{TIMEOUT}];
(
  node["amenity"="toilets"]{bbox_str};
  way["amenity"="toilets"]{bbox_str};
  relation["amenity"="toilets"]{bbox_str};
);
out center;
"""
    data = query_overpass(q)
    for el in data.get("elements", []):
        key = (el["type"], el["id"])
        if key in seen:
            continue
        seen.add(key)
        lat, lon = element_latlon(el)
        if lat is None or lon is None:
            continue
        tags = el.get("tags",{})
        feat = {
            "type":"Feature",
            "geometry":{"type":"Point","coordinates":[lon,lat]},
            "properties":{
                "osm_id":el["id"],
                "osm_type":el["type"],
                "name":tags.get("name"),
                "tags":tags
            }
        }
        features.append(feat)
    time.sleep(SLEEP)

geo = {"type":"FeatureCollection","features":features}
out_file = OUT_DIR/"toilets.geojson"
with open(out_file,"w",encoding="utf-8") as f:
    json.dump(geo,f,ensure_ascii=False,indent=2)

print(f"💾 Lưu {out_file} với {len(features)} toilets.")


🔍 Query toilets công cộng tại bbox (20.9,105.7,21.0,105.8)
🔍 Query toilets công cộng tại bbox (20.9,105.8,21.0,105.9)
⚠️ Lỗi server https://overpass.kumi.systems/api/interpreter: Expecting value: line 1 column 1 (char 0)
🔍 Query toilets công cộng tại bbox (20.9,105.9,21.0,106.0)
🔍 Query toilets công cộng tại bbox (21.0,105.7,21.099999999999998,105.8)
🔍 Query toilets công cộng tại bbox (21.0,105.8,21.099999999999998,105.9)
🔍 Query toilets công cộng tại bbox (21.0,105.9,21.099999999999998,106.0)
🔍 Query toilets công cộng tại bbox (21.099999999999998,105.7,21.2,105.8)
🔍 Query toilets công cộng tại bbox (21.099999999999998,105.8,21.2,105.9)
🔍 Query toilets công cộng tại bbox (21.099999999999998,105.9,21.2,106.0)
💾 Lưu opendata_hanoi/toilets.geojson với 141 toilets.


In [32]:
# Push lên GitHub
push_to_github(out_file)

/tmp/ipython-input-1839916761.py:54: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


✅ Đã push file toilets.geojson lên GitHub repo honganhss/OpenDataFitHou (branch main)


In [33]:
features = []
seen = set()

for bbox in BBOXES:
    bbox_str = f"({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]})"
    print(f"🔍 Query atm tại bbox {bbox_str}")
    q = f"""
[out:json][timeout:{TIMEOUT}];
(
  node["amenity"="atm"]{bbox_str};
  way["amenity"="atm"]{bbox_str};
  relation["amenity"="atm"]{bbox_str};
);
out center;
"""
    data = query_overpass(q)
    for el in data.get("elements", []):
        key = (el["type"], el["id"])
        if key in seen:
            continue
        seen.add(key)
        lat, lon = element_latlon(el)
        if lat is None or lon is None:
            continue
        tags = el.get("tags",{})
        feat = {
            "type":"Feature",
            "geometry":{"type":"Point","coordinates":[lon,lat]},
            "properties":{
                "osm_id":el["id"],
                "osm_type":el["type"],
                "name":tags.get("name"),
                "tags":tags
            }
        }
        features.append(feat)
    time.sleep(SLEEP)

geo = {"type":"FeatureCollection","features":features}
out_file = OUT_DIR/"atm.geojson"
with open(out_file,"w",encoding="utf-8") as f:
    json.dump(geo,f,ensure_ascii=False,indent=2)

print(f"💾 Lưu {out_file} với {len(features)} atm.")


🔍 Query atm tại bbox (20.9,105.7,21.0,105.8)
🔍 Query atm tại bbox (20.9,105.8,21.0,105.9)
🔍 Query atm tại bbox (20.9,105.9,21.0,106.0)
🔍 Query atm tại bbox (21.0,105.7,21.099999999999998,105.8)
🔍 Query atm tại bbox (21.0,105.8,21.099999999999998,105.9)
🔍 Query atm tại bbox (21.0,105.9,21.099999999999998,106.0)
🔍 Query atm tại bbox (21.099999999999998,105.7,21.2,105.8)
🔍 Query atm tại bbox (21.099999999999998,105.8,21.2,105.9)
🔍 Query atm tại bbox (21.099999999999998,105.9,21.2,106.0)
💾 Lưu opendata_hanoi/atm.geojson với 1416 atm.


In [34]:
# Push lên GitHub
push_to_github(out_file)

/tmp/ipython-input-1839916761.py:54: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


✅ Đã push file atm.geojson lên GitHub repo honganhss/OpenDataFitHou (branch main)


In [35]:
features = []
seen = set()

for bbox in BBOXES:
    bbox_str = f"({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]})"
    print(f"🔍 Query drinking_water tại bbox {bbox_str}")
    q = f"""
[out:json][timeout:{TIMEOUT}];
(
  node["amenity"="drinking_water"]{bbox_str};
  way["amenity"="drinking_water"]{bbox_str};
  relation["amenity"="drinking_water"]{bbox_str};
);
out center;
"""
    data = query_overpass(q)
    for el in data.get("elements", []):
        key = (el["type"], el["id"])
        if key in seen:
            continue
        seen.add(key)
        lat, lon = element_latlon(el)
        if lat is None or lon is None:
            continue
        tags = el.get("tags",{})
        feat = {
            "type":"Feature",
            "geometry":{"type":"Point","coordinates":[lon,lat]},
            "properties":{
                "osm_id":el["id"],
                "osm_type":el["type"],
                "name":tags.get("name"),
                "tags":tags
            }
        }
        features.append(feat)
    time.sleep(SLEEP)

geo = {"type":"FeatureCollection","features":features}
out_file = OUT_DIR/"drinking_water.geojson"
with open(out_file,"w",encoding="utf-8") as f:
    json.dump(geo,f,ensure_ascii=False,indent=2)

print(f"💾 Lưu {out_file} với {len(features)} drinking_water.")


🔍 Query drinking_water tại bbox (20.9,105.7,21.0,105.8)
🔍 Query drinking_water tại bbox (20.9,105.8,21.0,105.9)
🔍 Query drinking_water tại bbox (20.9,105.9,21.0,106.0)
🔍 Query drinking_water tại bbox (21.0,105.7,21.099999999999998,105.8)
🔍 Query drinking_water tại bbox (21.0,105.8,21.099999999999998,105.9)
🔍 Query drinking_water tại bbox (21.0,105.9,21.099999999999998,106.0)
🔍 Query drinking_water tại bbox (21.099999999999998,105.7,21.2,105.8)
🔍 Query drinking_water tại bbox (21.099999999999998,105.8,21.2,105.9)
🔍 Query drinking_water tại bbox (21.099999999999998,105.9,21.2,106.0)
💾 Lưu opendata_hanoi/drinking_water.geojson với 19 drinking_water.


In [36]:
# Push lên GitHub
push_to_github(out_file)

/tmp/ipython-input-1839916761.py:54: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


✅ Đã push file drinking_water.geojson lên GitHub repo honganhss/OpenDataFitHou (branch main)


In [37]:
features = []
seen = set()

for bbox in BBOXES:
    bbox_str = f"({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]})"
    print(f"🔍 Query hospital tại bbox {bbox_str}")
    q = f"""
[out:json][timeout:{TIMEOUT}];
(
  node["amenity"="hospital"]{bbox_str};
  way["amenity"="hospital"]{bbox_str};
  relation["amenity"="hospital"]{bbox_str};
);
out center;
"""
    data = query_overpass(q)
    for el in data.get("elements", []):
        key = (el["type"], el["id"])
        if key in seen:
            continue
        seen.add(key)
        lat, lon = element_latlon(el)
        if lat is None or lon is None:
            continue
        tags = el.get("tags",{})
        feat = {
            "type":"Feature",
            "geometry":{"type":"Point","coordinates":[lon,lat]},
            "properties":{
                "osm_id":el["id"],
                "osm_type":el["type"],
                "name":tags.get("name"),
                "tags":tags
            }
        }
        features.append(feat)
    time.sleep(SLEEP)

geo = {"type":"FeatureCollection","features":features}
out_file = OUT_DIR/"hospital.geojson"
with open(out_file,"w",encoding="utf-8") as f:
    json.dump(geo,f,ensure_ascii=False,indent=2)

print(f"💾 Lưu {out_file} với {len(features)} hospital.")


🔍 Query hospital tại bbox (20.9,105.7,21.0,105.8)
🔍 Query hospital tại bbox (20.9,105.8,21.0,105.9)
🔍 Query hospital tại bbox (20.9,105.9,21.0,106.0)
🔍 Query hospital tại bbox (21.0,105.7,21.099999999999998,105.8)
🔍 Query hospital tại bbox (21.0,105.8,21.099999999999998,105.9)
🔍 Query hospital tại bbox (21.0,105.9,21.099999999999998,106.0)
🔍 Query hospital tại bbox (21.099999999999998,105.7,21.2,105.8)
⚠️ Lỗi server https://overpass.kumi.systems/api/interpreter: Expecting value: line 1 column 1 (char 0)
🔍 Query hospital tại bbox (21.099999999999998,105.8,21.2,105.9)
🔍 Query hospital tại bbox (21.099999999999998,105.9,21.2,106.0)
💾 Lưu opendata_hanoi/hospital.geojson với 157 hospital.


In [38]:
# Push lên GitHub
push_to_github(out_file)

/tmp/ipython-input-1839916761.py:54: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


✅ Đã push file hospital.geojson lên GitHub repo honganhss/OpenDataFitHou (branch main)


In [39]:
features = []
seen = set()

for bbox in BBOXES:
    bbox_str = f"({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]})"
    print(f"🔍 Query school tại bbox {bbox_str}")
    q = f"""
[out:json][timeout:{TIMEOUT}];
(
  node["amenity"="school"]{bbox_str};
  way["amenity"="school"]{bbox_str};
  relation["amenity"="school"]{bbox_str};
);
out center;
"""
    data = query_overpass(q)
    for el in data.get("elements", []):
        key = (el["type"], el["id"])
        if key in seen:
            continue
        seen.add(key)
        lat, lon = element_latlon(el)
        if lat is None or lon is None:
            continue
        tags = el.get("tags",{})
        feat = {
            "type":"Feature",
            "geometry":{"type":"Point","coordinates":[lon,lat]},
            "properties":{
                "osm_id":el["id"],
                "osm_type":el["type"],
                "name":tags.get("name"),
                "tags":tags
            }
        }
        features.append(feat)
    time.sleep(SLEEP)

geo = {"type":"FeatureCollection","features":features}
out_file = OUT_DIR/"school.geojson"
with open(out_file,"w",encoding="utf-8") as f:
    json.dump(geo,f,ensure_ascii=False,indent=2)

print(f"💾 Lưu {out_file} với {len(features)} school.")


🔍 Query school tại bbox (20.9,105.7,21.0,105.8)
🔍 Query school tại bbox (20.9,105.8,21.0,105.9)
🔍 Query school tại bbox (20.9,105.9,21.0,106.0)
🔍 Query school tại bbox (21.0,105.7,21.099999999999998,105.8)
🔍 Query school tại bbox (21.0,105.8,21.099999999999998,105.9)
🔍 Query school tại bbox (21.0,105.9,21.099999999999998,106.0)
🔍 Query school tại bbox (21.099999999999998,105.7,21.2,105.8)
🔍 Query school tại bbox (21.099999999999998,105.8,21.2,105.9)
🔍 Query school tại bbox (21.099999999999998,105.9,21.2,106.0)
💾 Lưu opendata_hanoi/school.geojson với 1038 school.


In [40]:
# Push lên GitHub
push_to_github(out_file)

/tmp/ipython-input-1839916761.py:54: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


✅ Đã push file school.geojson lên GitHub repo honganhss/OpenDataFitHou (branch main)


In [41]:
# Get the path of the current notebook
notebook_path = !echo "$PWD"/*.ipynb
if notebook_path:
    notebook_file = Path(notebook_path[0])
    print(f"🔍 Found notebook: {notebook_file}")
    # Push the notebook to GitHub
    push_to_github(notebook_file)
else:
    print("❌ Could not find the notebook file.")

🔍 Found notebook: /content/*.ipynb
❌ File /content/*.ipynb không tồn tại.
